# KBG Syndrome

Data from [Martinez-Cayuelas E, et al. Clinical description, molecular delineation and genotype-phenotype correlation in 340 patients with KBG syndrome: addition of 67 new patients](https://pubmed.ncbi.nlm.nih.gov/36446582).

We investigate subjects with mutations in *ANKRD11*.

In [1]:
import genophenocorr

print(f"Using genophenocorr version {genophenocorr.__version__}")

Using genophenocorr version 0.1.1dev


## Setup

### Load HPO

We use HPO `v2023-10-09` release for this analysis.

In [2]:
import hpotk

store = hpotk.configure_ontology_store()
hpo = store.load_minimal_hpo(release='v2023-10-09')
print(f'Loaded HPO v{hpo.version}')

Loaded HPO v2023-10-09


### Load Phenopackets

We will load phenopacket JSON files located in `phenopackets` folder that is next to the notebook.

In [3]:
from genophenocorr.preprocessing import configure_caching_cohort_creator, load_phenopacket_folder

fpath_phenopackets = 'phenopackets'
cohort_creator = configure_caching_cohort_creator(hpo, timeout=20)
cohort = load_phenopacket_folder(fpath_phenopackets, cohort_creator)

Patients Created:  16%|█▋        | 54/328 [00:09<00:55,  4.96it/s]Expected a result but got an Error for variant: 16_89280752_89280752_G_T
{"error":"Could not connect to database homo_sapiens_core_111_38 as user ensro using [DBI:mysql:database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571] as a locator:DBI connect('database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571','ensro',...) failed: Can't connect to MySQL server on 'fb1-mysql-ens-rest-web.ebi.ac.uk' (111) at /nfs/public/ro/ensweb/live/rest/www_111/ensembl/modules/Bio/EnsEMBL/DBSQL/DBConnection.pm line 260."}
Patients Created:  24%|██▍       | 78/328 [00:29<03:01,  1.38it/s]Expected a result but got an Error for variant: 16_89284779_89284779_G_T
{"error":"Could not connect to database homo_sapiens_core_111_38 as user ensro using [DBI:mysql:database=homo_sapiens_core_111_38;host=fb1-mysql-ens-rest-web.ebi.ac.uk;port=4571] as a locator:DBI connect('database=homo_sapiens_core_11

### Pick transcript

We choose the [MANE Select](https://www.ncbi.nlm.nih.gov/nuccore/NM_013275.6) transcript for *ANKRD11*.

In [4]:
tx_id = 'NM_013275.6'

## Explore cohort

Explore the cohort to guide selection of the genotype-phenotype analysis.


In [5]:
from IPython.display import HTML, display
from genophenocorr.view import CohortViewer

viewer = CohortViewer(hpo)

In [6]:
display(HTML(viewer.cohort_summary_table(cohort)))

In [7]:
display(HTML(viewer.hpo_term_counts_table(cohort))) ## Add Labels to output

HPO Term,Count
Macrodontia (HP:0001572),149
Abnormality of the hand (HP:0001155),129
Intellectual disability (HP:0001249),126
Global developmental delay (HP:0001263),105
Abnormal external nose morphology (HP:0010938),88
Short stature (HP:0004322),88
Thick eyebrow (HP:0000574),85
Long philtrum (HP:0000343),81
Hearing impairment (HP:0000365),56
Triangular face (HP:0000325),53


In [8]:
display(HTML(viewer.variants_table(cohort, tx_id))) 

Variant,Effect,Count,Key
c.1903_1907del,FRAMESHIFT_VARIANT,33,16_89284634_89284639_GTGTTT_G
c.2408_2412del,FRAMESHIFT_VARIANT,10,16_89284129_89284134_CTTTTT_C
c.1381_1384del,FRAMESHIFT_VARIANT,8,16_89285157_89285161_GTTTC_G
c.2398_2401del,FRAMESHIFT_VARIANT,8,16_89284140_89284144_TTTTC_T
c.6792_6793insC,FRAMESHIFT_VARIANT,5,16_89279749_89279749_C_CG
c.7481_7482insC,FRAMESHIFT_VARIANT,5,16_89275180_89275180_A_AG
c.2175_2178del,FRAMESHIFT_VARIANT,3,16_89284363_89284367_CTTTG_C
c.2182_2183del,FRAMESHIFT_VARIANT,3,16_89284358_89284360_GAT_G
c.3224_3227del,FRAMESHIFT_VARIANT,3,16_89283314_89283318_CCTTT_C
c.4406G>A,STOP_GAINED,3,16_89282136_89282136_C_T


## Configure the analysis

In [10]:
from genophenocorr.analysis import configure_cohort_analysis, CohortAnalysisConfiguration
from genophenocorr.analysis.predicate import PatientCategories

analysis_config = CohortAnalysisConfiguration()
analysis_config.missing_implies_excluded = True
analysis_config.pval_correction = 'fdr_bh'
analysis_config.min_perc_patients_w_hpo = 0.1
analysis = configure_cohort_analysis(cohort, hpo, config=analysis_config)

Test for presence of genotype-phenotype correlations between frameshift variants vs. others.

In [11]:
from genophenocorr.model import VariantEffect

frameshift = analysis.compare_by_variant_effect(VariantEffect.FRAMESHIFT_VARIANT, tx_id)
frameshift.summarize(hpo, PatientCategories.YES)

FRAMESHIFT_VARIANT on NM_013275.6                       Yes             No  \
                                                      Count Percent  Count   
Intellectual disability [HP:0001249]                 84/141     60%  42/52   
Abnormality of mental function [HP:0011446]          96/141     68%  46/53   
Abnormal external nose morphology [HP:0010938]       57/125     46%  31/46   
Abnormal nasal morphology [HP:0005105]               57/125     46%  31/46   
Abnormality of the nose [HP:0000366]                 57/125     46%  31/46   
Neurodevelopmental abnormality [HP:0012759]         107/142     75%  50/55   
Global developmental delay [HP:0001263]              71/121     59%  34/44   
Neurodevelopmental delay [HP:0012758]                71/121     59%  34/44   
Abnormal nervous system physiology [HP:0012638]     113/142     80%  52/56   
Abnormality of the nervous system [HP:0000707]      113/142     80%  52/56   
Abnormal upper lip morphology [HP:0000177]           54/128     42%  27/46   
Abnormal lip morphology [HP:0000159]                 54/128     42%  27/46   
Abnormality of the philtrum [HP:0000288]             54/128     42%  27/46   
Long philtrum [HP:0000343]                           54/128     42%  27/46   
Abnormality of the upper limb [HP:0002817]           86/139     62%  43/56   
Abnormality of the hand [HP:0001155]                 86/139     62%  43/56   
Abnormality of limbs [HP:0040064]                    86/139     62%  43/56   
Abnormal eyebrow morphology [HP:0000534]             57/124     46%  28/46   
Abnormal ocular adnexa morphology [HP:0030669]       57/124     46%  28/46   
Thick eyebrow [HP:0000574]                           57/124     46%  28/46   
Abnormality of the orbital region [HP:0000315]       57/124     46%  28/46   
Abnormality of the integument [HP:0001574]           57/124     46%  28/46   
Abnormality of the ocular adnexa [HP:0032039]        57/124     46%  28/46   
Abnormal hair morphology [HP:0001595]                57/124     46%  28/46   
Abnormal skin adnexa morphology [HP:0011138]         57/124     46%  28/46   
Abnormality of the outer ear [HP:0000356]            31/117     26%  15/42   
Abnormal ear morphology [HP:0031703]                 31/117     26%  15/42   
Triangular face [HP:0000325]                         36/116     31%  17/43   
Abnormal facial shape [HP:0001999]                   36/116     31%  17/43   
Abnormal oral morphology [HP:0031816]               118/139     85%  50/56   
Abnormal oral cavity morphology [HP:0000163]        118/139     85%  50/56   
Abnormality of the mouth [HP:0000153]               118/139     85%  50/56   
Short attention span [HP:0000736]                    27/113     24%  11/38   
Recurrent maladaptive behavior [HP:5200241]          27/113     24%  11/38   
Hyperactivity [HP:0000752]                           27/113     24%  11/38   
Cognitive impairment [HP:0100543]                    27/113     24%  11/38   
Reduced attention regulation [HP:5200044]            27/113     24%  11/38   
Attention deficit hyperactivity disorder [HP:00...   27/113     24%  11/38   
Disinhibition [HP:0000734]                           27/113     24%  11/38   
Atypical behavior [HP:0000708]                       37/114     32%  15/39   
Phenotypic abnormality [HP:0000118]                 142/145     98%  58/58   
All [HP:0000001]                                    142/145     98%  58/58   
Abnormality of head or neck [HP:0000152]            127/140     91%  54/57   
Abnormality of the head [HP:0000234]                127/140     91%  54/57   
Abnormality of the face [HP:0000271]                127/140     91%  54/57   
Growth delay [HP:0001510]                            62/132     47%  26/51   
Short stature [HP:0004322]                           62/132     47%  26/51   
Growth abnormality [HP:0001507]                      62/132     47%  26/51   
Abnormality of body height [HP:0000002]              62/132     47%  26/51   
Autistic behavior [HP:

Test for presence of genotype-phenotype correlations between subjects with >=1 allele of a variant vs. the other subjects:


In [12]:
var_single = analysis.compare_by_variant_key('16_89284634_89284639_GTGTTT_G')
var_single.summarize(hpo, PatientCategories.YES)

>=1 allele of the variant 16_89284634_89284639_GTGTTT_G    Yes          \
                                                         Count Percent   
Neurodevelopmental abnormality [HP:0012759]              21/33     64%   
Intellectual disability [HP:0001249]                     15/32     47%   
Abnormality of head or neck [HP:0000152]                 27/33     82%   
Abnormality of the head [HP:0000234]                     27/33     82%   
Abnormality of the face [HP:0000271]                     27/33     82%   
Abnormal nervous system physiology [HP:0012638]          23/33     70%   
Abnormality of the nervous system [HP:0000707]           23/33     70%   
Abnormality of the ear [HP:0000598]                      10/28     36%   
Abnormal eyebrow morphology [HP:0000534]                 10/30     33%   
Abnormal ocular adnexa morphology [HP:0030669]           10/30     33%   
Thick eyebrow [HP:0000574]                               10/30     33%   
Abnormality of the orbital region [HP:0000315]           10/30     33%   
Abnormality of the integument [HP:0001574]               10/30     33%   
Abnormality of the ocular adnexa [HP:0032039]            10/30     33%   
Abnormal hair morphology [HP:0001595]                    10/30     33%   
Abnormal skin adnexa morphology [HP:0011138]             10/30     33%   
Phenotypic abnormality [HP:0000118]                      31/33     94%   
All [HP:0000001]                                         31/33     94%   
Abnormality of mental function [HP:0011446]              19/32     59%   
Global developmental delay [HP:0001263]                  14/29     48%   
Neurodevelopmental delay [HP:0012758]                    14/29     48%   
Abnormal oral morphology [HP:0031816]                    25/33     76%   
Abnormal oral cavity morphology [HP:0000163]             25/33     76%   
Abnormality of the mouth [HP:0000153]                    25/33     76%   
Triangular face [HP:0000325]                              6/28     21%   
Abnormal facial shape [HP:0001999]                        6/28     21%   
Autistic behavior [HP:0000729]                            3/27     11%   
Abnormal ear physiology [HP:0031704]                      7/28     25%   
Hearing impairment [HP:0000365]                           7/28     25%   
Hearing abnormality [HP:0000364]                          7/28     25%   
Abnormal upper lip morphology [HP:0000177]               11/30     37%   
Abnormal lip morphology [HP:0000159]                     11/30     37%   
Abnormality of the philtrum [HP:0000288]                 11/30     37%   
Long philtrum [HP:0000343]                               11/30     37%   
Abnormal external nose morphology [HP:0010938]           13/31     42%   
Abnormal nasal morphology [HP:0005105]                   13/31     42%   
Abnormality of the nose [HP:0000366]                     13/31     42%   
Abnormality of the outer ear [HP:0000356]                 6/28     21%   
Abnormal ear morphology [HP:0031703]                      6/28     21%   
Macrodontia [HP:0001572]                                 23/31     74%   
Abnormal dental morphology [HP:0006482]                  23/31     74%   
Abnormality of the dentition [HP:0000164]                23/31     74%   
Atypical behavior [HP:0000708]                            8/27     30%   
Growth delay [HP:0001510]                                14/32     44%   
Short stature [HP:0004322]                               14/32     44%   
Growth abnormality [HP:0001507]                          14/32     44%   
Abnormality of body height [HP:0000002]                  14/32     44%   
Abnormality of the upper limb [HP:0002817]               22/33     67%   
Short attention span [HP:0000736]                         7/27     26%   
Recurrent maladaptive behavior [HP:5200241]               7/27     26%   
Abnormality of the hand [HP:0001155]                     22/33     67%   
Hyperactivity [HP:0000752]                                7/27     26%   
Cogn

Or between subjects with one variant vs. the other variant.

In [13]:
var_double = analysis.compare_by_variant_keys('16_89284129_89284134_CTTTTT_C', '16_89284634_89284639_GTGTTT_G')
var_double.summarize(hpo, PatientCategories.YES)

>=1 allele of either variant 16_89284129_89284134_CTTTTT_C or variant 16_89284634_89284639_GTGTTT_G  First  \
                                                                                                     Count   
Short attention span [HP:0000736]                                                                      0/8   
Recurrent maladaptive behavior [HP:5200241]                                                            0/8   
Hyperactivity [HP:0000752]                                                                             0/8   
Cognitive impairment [HP:0100543]                                                                      0/8   
Reduced attention regulation [HP:5200044]                                                              0/8   
Attention deficit hyperactivity disorder [HP:00...                                                     0/8   
Disinhibition [HP:0000734]                                                                             0/8   
Growth delay [HP:0001510]                                                                              5/8   
Short stature [HP:0004322]                                                                             5/8   
Growth abnormality [HP:0001507]                                                                        5/8   
Abnormality of body height [HP:0000002]                                                                5/8   
Autistic behavior [HP:0000729]                                                                         2/9   
Abnormal ear physiology [HP:0031704]                                                                   3/8   
Hearing impairment [HP:0000365]                                                                        3/8   
Hearing abnormality [HP:0000364]                                                                       3/8   
Abnormality of the ear [HP:0000598]                                                                    4/8   
Abnormality of the upper limb [HP:0002817]                                                            8/10   
Abnormality of the hand [HP:0001155]                                                                  8/10   
Abnormality of limbs [HP:0040064]                                                                     8/10   
Abnormal external nose morphology [HP:0010938]                                                         5/9   
Abnormal nasal morphology [HP:0005105]                                                                 5/9   
Abnormality of the nose [HP:0000366]                                                                   5/9   
Abnormal upper lip morphology [HP:0000177]                                                             4/9   
Abnormal lip morphology [HP:0000159]                                                                   4/9   
Abnormality of the philtrum [HP:0000288]                                                               4/9   
Long philtrum [HP:0000343]                                                                             4/9   
Abnormal nervous system physiology [HP:0012638]                                                       7/10   
Abnormal oral morphology [HP:0031816]                                                                 8/10   
Abnormal eyebrow morphology [HP:0000534]                                                               3/9   
Abnormal ocular adnexa morphology [HP:0030669]                                                         3/9   
Global developmental delay [HP:0001263]                                                                4/9   
Abnormality of the nervous system [HP:0000707]                                                        7/10   
Intellectual disability [HP:0001249]                                                                  5/10   
Abnormality of head or neck [HP:0000152]                                                              9/10   
Abnormality of the outer ear [HP:0000356

TODO - finalize!